# Pub accessibility for Inner London

## Data and preparation

> Pubs (data provided by CAMRA in January 2019)
> https://data.london.gov.uk/download/cultural-infrastructure-map/7e6b0f36-da01-4029-99d7-640d89c4a0a4/Pubs.csv
> Buildings defined by CAMRA as pubs. Licensed premises must be open to and welcome the general public without requiring membership or residency, allow free entry (1), serve at least one draught beer (2), allow drinking without requiring food to be consumed, have at least one indoor area not laid out for meals, and permit drinks to be purchased in person at a bar (3), without relying on table service.’ (1) Except when entertainment is provided on limited occasions (2) Includes cask or keg beer (3) Includes also a hatch or specific service point’
> How data was compiled: Data provided by the Campaign for Real Ale.
> Credit for use: The Campaign for Real Ale

This examples calculates landuse accessibility to pubs for inner London. For additional information on network preparation, see the corresponding notebook on [graph cleaning](https://colab.research.google.com/github/cityseer/cityseer/blob/master/demo_notebooks/graph_cleaning.ipynb) for more information.

This example will make use of OSM data downloaded from the [OSM API](https://wiki.openstreetmap.org/wiki/API) for a 5000m radius around London Soho.

In [1]:
%matplotlib inline
!pip install cityseer
!pip install pyproj

In [2]:
from pyproj import CRS, Transformer
from shapely import geometry
import utm

# starting point
lng, lat = -0.13396079424572427, 51.51371088849723
distance = 5000
easting, northing, utm_zone_number, utm_zone_letter = utm.from_latlon(lat, lng)
centre = geometry.Point(easting, northing)
# CRS transformations
crs_utm = CRS.from_dict({'proj': 'utm', 'zone': utm_zone_number, 'south': False})
crs_bng = CRS.from_epsg(27700)
crs_transformer = Transformer.from_crs(crs_bng, crs_utm, always_xy=True)

In [ ]:
# prepare network from OSM
from cityseer.tools import graphs, mock
from cityseer.metrics import networks

G_utm = mock.make_buffered_osm_graph(lng, lat, distance)

G = graphs.nX_simple_geoms(G_utm)
G = graphs.nX_remove_filler_nodes(G)
G = graphs.nX_remove_dangling_nodes(G,
                                    despine=20,
                                    remove_disconnected=True)
G = graphs.nX_remove_filler_nodes(G)
G1 = graphs.nX_consolidate_nodes(G,
                                 buffer_dist=10,
                                 min_node_group=3)
G2 = graphs.nX_split_opposing_geoms(G1,
                                    buffer_dist=15)
G3 = graphs.nX_consolidate_nodes(G2,
                                 buffer_dist=15,
                                 crawl=False,
                                 min_node_degree=2,
                                 cent_min_degree=4)
G_decomp = graphs.nX_decompose(G3, 25)
N = networks.NetworkLayerFromNX(G_decomp,
                                distances=[400])

In [ ]:
# prepare data layer
import pandas as pd
import numpy as np
from cityseer.metrics import layers

# load data
pubs_data = pd.read_csv('https://data.london.gov.uk/download/cultural-infrastructure-map/7e6b0f36-da01-4029-99d7-640d89c4a0a4/Pubs.csv')
pubs_data_filtered = pubs_data.loc[np.logical_and(~np.isnan(pubs_data.easting),
                                                  ~np.isnan(pubs_data.northing))]
# unpacking to a dictionary for convenience
labels = []
fd = {}
for idx, data in pubs_data_filtered.iterrows():
    x, y = crs_transformer.transform(data.easting, data.northing)
    loc = geometry.Point(x, y)
    if centre.distance(loc) < distance:
        labels.append('pub')
        fd[idx] = {'x': x, 'y': y}
DL = layers.DataLayerFromDict(fd)
# assign trees to network
DL.assign_to_network(Network_Layer=N, max_dist=300)
# in this case, all labels are the same as all points are the same data type
DL.compute_accessibilities(landuse_labels=labels,
                           accessibility_keys=['pub'])

In [ ]:
# data can be extracted, plotted, or otherwise manipulated directly from the numpy arrays
# see the getting started guide if you'd prefer to first convert back to NetworkX
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import minmax_scale


def simple_plot(xs: np.ndarray,
                ys: np.ndarray,
                vals: np.ndarray):
    """ """
    # extents
    easting, northing = utm.from_latlon(lat, lng)[:2]
    buff = geometry.Point(easting, northing).buffer(3500)
    min_x, min_y, max_x, max_y = buff.bounds
    # fig
    fig, ax = plt.subplots(1, 1, figsize=(8, 8), dpi=200, facecolor='white')
    # filter
    select = xs > min_x
    select = np.logical_and(select, xs < max_x)
    select = np.logical_and(select, ys > min_y)
    select = np.logical_and(select, ys < max_y)
    select_idx = np.where(select)[0]
    # remove any extreme outliers
    v = np.clip(vals, np.nanpercentile(vals, .01), np.nanpercentile(vals, 99.9))
    # shape if wanted
    c = v ** 1
    c = minmax_scale(c, feature_range=(0, 1))
    s = v ** 1
    s = minmax_scale(s, feature_range=(0, 1.5))
    # plot
    ax.scatter(xs[select_idx],
               ys[select_idx],
               c=c[select_idx],
               s=s[select_idx],
               linewidths=0,
               edgecolors='none',
               cmap='Greens')
    # limits
    ax.set_xlim(left=min_x, right=max_x)
    ax.set_ylim(bottom=min_y, top=max_y)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_aspect(1)
    ax.set_facecolor('white')
    plt.show()

simple_plot(N.node_x_arr, N.node_y_arr, N.metrics['accessibility']['weighted']['pub'][400])